In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

%cd drive/MyDrive/Colab_Notebooks/exp4/
%pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/exp4


'/content/drive/MyDrive/Colab_Notebooks/exp4'

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import time
import random
import warnings
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from gensim.models import FastText
import os
warnings.filterwarnings("ignore")

In [ ]:
m_c = 16255

train_csv = pd.read_csv('./train.csv')
val_csv = pd.read_csv('val.csv')
test_csv = pd.read_csv('test.csv')

train = train_csv[(train_csv['id']<m_c)].values.tolist()
val = val_csv[(val_csv['id']<m_c)].values.tolist()

In [ ]:
print(train[1])
print(val[1])
print(len(train))
print(len(val))

In [ ]:
def splitit(train_txt):
    texts = []
    for tx in train_txt:
        l = tx[2].replace(';', "; ")
        l = re.sub('(?<! )(?=[.,!?;()])|(?<=[.,!?;()])(?! )', r' ', l)
        texts.append(l)
    return texts

In [ ]:
import re

# all_txt = train_txt + val_txt + test_txt
x_train = splitit(train)
print(x_train[100])
x_val = splitit(val)

In [ ]:
y_train = []
for l in train:
    y_train.append(l[3])

y_val = []
for l in val:
    y_val.append(l[3])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv_fit = cv.fit_transform(x_train)
print(cv_fit)

  (0, 419)	1
  (0, 474)	1
  (0, 3503)	2
  (0, 4477)	2
  (0, 6301)	1
  (0, 4236)	1
  (0, 881)	1
  (0, 3892)	1
  (1, 419)	2
  (1, 3503)	1
  (1, 4477)	2
  (1, 188)	1
  (1, 882)	1
  (1, 2270)	1
  (1, 289)	3
  (1, 5695)	1
  (1, 2380)	1
  (1, 802)	1
  (1, 6094)	2
  (2, 419)	5
  (2, 3503)	1
  (2, 4477)	2
  (2, 882)	1
  (2, 2270)	1
  (2, 289)	6
  :	:
  (10301, 4236)	1
  (10301, 882)	1
  (10301, 2270)	1
  (10301, 289)	2
  (10301, 5695)	1
  (10301, 2380)	1
  (10301, 1167)	2
  (10301, 1821)	1
  (10301, 3036)	1
  (10301, 200)	1
  (10301, 1168)	1
  (10301, 4198)	1
  (10301, 5164)	1
  (10301, 2660)	1
  (10302, 3503)	1
  (10302, 4477)	2
  (10302, 882)	1
  (10302, 2270)	1
  (10302, 2380)	1
  (10302, 1975)	3
  (10302, 1247)	1
  (10302, 5001)	1
  (10302, 163)	1
  (10302, 1384)	1
  (10302, 2028)	1


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

#3.进行特征抽取
tf = TfidfVectorizer()
X_train = tf.fit_transform(x_train)
X_val = tf.transform(x_val)
#4.进行朴素贝叶斯算法分类
bayes = MultinomialNB(alpha=0.1)
bayes.fit(X_train,y_train)
score = bayes.score(X_val,y_val)
print("Bayes模型的验证集预测准确率为：",score)
print("模型集成的验证集预测准确率为：",(score * 1145 + 1255)/2400)

Bayes模型的验证集预测准确率为： 0.8139737991266376
模型集成的验证集预测准确率为： 0.91125


In [ ]:
test_less = test_csv[(test_csv['id']<m_c)].values.tolist()
x_test = splitit(test_less)

In [ ]:
X_test = tf.transform(x_test)
y_predict = bayes.predict(X_test)
print("测试集的预测结果为：",y_predict)

测试集的预测结果为： [0 0 1 ... 0 0 0]


In [ ]:
ans = []
ind = test_csv[['id']].values.tolist()
j = 0
for i in ind:
    if i[0] >= m_c:
        ans.append([i[0], 3])
    else:
        ans.append([i[0], y_predict[j]])
        j += 1

In [ ]:
df = pd.DataFrame(ans)
df.to_csv('./ans.csv')

In [ ]:
def load_data():
    words = []
    y_train = []
    with open('20news.json', 'r') as f:
        lines = []
        for l in f:
            lines.append(l)
        random.shuffle(lines)
        for l in lines:
            data = json.loads(l)
            words.append(data["text"])
            onehot = [0 for i in range(20)]
            onehot[data["label"]] = 1
            y_train.append(onehot)
    model = FastText.load('fasttext.model')
    len1 = len(words)
    labels_maxn = 20
    words_maxn = 32
    dict_vec={}
    for word in model.wv.index2word:
        dict_vec[word] = list(model.wv[word])
    x_train=[]
    maxn = 340

    zerolist = [0 for i in range(labels_maxn)]
    for i in range(len1):
        if y_train[i] == zerolist:
            continue
        embedding_one=[]
        words_one=words[i]
        len_word = len(words_one)
        if len_word >= maxn:
            words_one = words_one[0:maxn]
            for j in range(maxn):
                if words_one[j] in dict_vec:
                    embedding_one.append(dict_vec[words_one[j]])
                else:
                    dict_vec[words_one[j]]=[0 for temp in range(words_maxn)]
                    embedding_one.append(dict_vec[words_one[j]])
        else:
            for k in range(maxn-len_word):
                words_one.append("#")
            for j in range(maxn):
                if words_one[j] in dict_vec:
                    embedding_one.append(dict_vec[words_one[j]])
                else:
                    dict_vec[words_one[j]] = [0 for temp in range(words_maxn)]
                    embedding_one.append(dict_vec[words_one[j]])
        x_train.append(embedding_one)

    y_train_suc = []
    for y_train_one in y_train:
        if y_train_one == zerolist:
            continue
        y_train_suc.append(y_train_one)
    y_train = y_train_suc

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    return x_train,  y_train

In [ ]:
x_all, y_all=load_data()

print(np.array(x_all).shape,flush=True)
print(np.array(y_all).shape,flush=True)
# 有效数据18828个
maxn = len(x_all)
trainn = int(maxn * 0.8)
x_train = x_all[0:trainn]
x_val = x_all[trainn:maxn]
y_train = y_all[0:trainn]
y_val = y_all[trainn:maxn]

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(input_dim=32, output_dim=256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(256, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(output_dim=20))
model.add(Activation("sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(x_train, y_train, batch_size=128, nb_epoch=10, validation_data=(x_val, y_val))
print(model.summary(),flush=True)

Train on 10303 samples, validate on 1145 samples
Epoch 1/10
10303/10303 [==============================] - 231s 20ms/step - loss: 0.2590 - val_loss: 0.2265
Epoch 2/10
10303/10303 [==============================] - 233s 21ms/step - loss: 0.2266 - val_loss: 0.2038
Epoch 3/10
10303/10303 [==============================] - 229s 20ms/step - loss: 0.1943 - val_loss: 0.1843
Epoch 4/10
10303/10303 [==============================] - 233s 21ms/step - loss: 0.1812 - val_loss: 0.1765
Epoch 5/10
10303/10303 [==============================] - 232s 21ms/step - loss: 0.1695 - val_loss: 0.1621
Epoch 6/10
10303/10303 [==============================] - 233s 21ms/step - loss: 0.1527 - val_loss: 0.1562
Epoch 7/10
10303/10303 [==============================] - 237s 21ms/step - loss: 0.1458 - val_loss: 0.1470
Epoch 8/10
10303/10303 [==============================] - 236s 21ms/step - loss: 0.1336 - val_loss: 0.1352
Epoch 9/10
10303/10303 [==============================] - 232s 21ms/step - loss: 0.1249 - val_l

In [ ]:
#进行测试并得到精度
predicted = model.predict(x_train)
predicted=predicted.tolist()
predicted_last=[]
for i in range(len(predicted)):
    ls = predicted[i]
    mx = 0
    for j in range(len(ls)):
        if ls[j]>ls[mx]:
            mx = j
    predicted_last.append(mx)
    
true_predict = 0
false_predict = 0
for i in range(len(y_train)):
    ls = y_train[i]
    label = predicted_last[i]
    if ls[label] == 1:
        true_predict += 1
    else:
        false_predict += 1

print(true_predict, false_predict)
print(true_predict/(true_predict+false_predict))

6948 3355
0.6743666893140139


In [ ]:
#进行测试并得到精度
predicted = model.predict(x_test)
predicted=predicted.tolist()
predicted_last=[]
for i in range(len(predicted)):
    ls = predicted[i]
    mx = 0
    for j in range(len(ls)):
        if ls[j]>ls[mx]:
            mx = j
    predicted_last.append(mx)

In [ ]:
true_predict = 0
false_predict = 0
for i in range(len(y_test)):
    ls = y_test[i]
    label = predicted_last[i]
    if ls[label] == 1:
        true_predict += 1
    else:
        false_predict += 1

print(true_predict, false_predict)
print(true_predict/(true_predict+false_predict))

636 509
0.5554585152846686
